In [1]:
%pip install statsforecast
%pip install statsmodels
%pip install pmdarima
%pip install pandas
%pip install plotly
%pip install nbformat>=4.2.0

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# YEARLY SUNSPOT DATA

In [2]:
from statsforecast.models import ARIMA
from pmdarima.arima import auto_arima
import plotly.graph_objs as go
import statsmodels.api as sm
from arima import Model

c:\Users\e10110199\rust_projects\arima\env\lib\site-packages\statsforecast\core.py:26: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
c:\Users\e10110199\rust_projects\arima\env\lib\site-packages\statsforecast\utils.py:237: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  "ds": pd.date_range(start="1949-01-01", periods=len(AirPassengers), freq="M"),


In [3]:
H = 45
DS = "YEAR"
Y = "SUNACTIVITY"

In [4]:
df = sm.datasets.sunspots.load_pandas().data
y_train = df[Y].values[:-H]

# Search hyper-parameters

In [17]:
search = auto_arima(
    y=y_train,
    start_p=0,
    max_p=5,
    d=0,
    max_d=1,
    start_q=0,
    max_q=5,
    start_P=0,
    max_P=2,
    start_Q=0,
    max_Q=2,
    D=1,
    seasonal=True,
    m=43,
    stepwise=True,
    maxiter=50,
    information_criterion="aic",
    suppress_warnings=True,
    trace=True
)

Performing stepwise search to minimize aic
 ARIMA(0,0,0)(0,1,0)[43] intercept   : AIC=2359.096, Time=0.21 sec
 ARIMA(1,0,0)(1,1,0)[43] intercept   : AIC=2078.131, Time=7.20 sec
 ARIMA(0,0,1)(0,1,1)[43] intercept   : AIC=2121.528, Time=10.46 sec
 ARIMA(0,0,0)(0,1,0)[43]             : AIC=2359.393, Time=0.08 sec
 ARIMA(1,0,0)(0,1,0)[43] intercept   : AIC=2107.876, Time=0.64 sec
 ARIMA(1,0,0)(2,1,0)[43] intercept   : AIC=2071.946, Time=19.71 sec
 ARIMA(1,0,0)(2,1,1)[43] intercept   : AIC=2073.859, Time=24.00 sec
 ARIMA(1,0,0)(1,1,1)[43] intercept   : AIC=2074.445, Time=8.92 sec
 ARIMA(0,0,0)(2,1,0)[43] intercept   : AIC=2317.910, Time=16.47 sec
 ARIMA(2,0,0)(2,1,0)[43] intercept   : AIC=1956.030, Time=24.01 sec
 ARIMA(2,0,0)(1,1,0)[43] intercept   : AIC=1960.115, Time=11.76 sec
 ARIMA(2,0,0)(2,1,1)[43] intercept   : AIC=inf, Time=63.67 sec
 ARIMA(2,0,0)(1,1,1)[43] intercept   : AIC=inf, Time=22.25 sec
 ARIMA(3,0,0)(2,1,0)[43] intercept   : AIC=1956.788, Time=29.49 sec
 ARIMA(2,0,1)(2,1,0)

In [18]:
params = search.get_params()
order = params["order"]
# order = (2, 0, 1)
# order = (3, 0, 0)
seasonal_order = params["seasonal_order"]
# seasonal_order = (2, 1, 0, 43)
params

{'maxiter': 50,
 'method': 'lbfgs',
 'order': (2, 0, 0),
 'out_of_sample_size': 0,
 'scoring': 'mse',
 'scoring_args': {},
 'seasonal_order': (1, 1, 1, 43),
 'start_params': None,
 'suppress_warnings': True,
 'trend': None,
 'with_intercept': False}

# Train arima models

### My Arima

In [23]:
%%timeit
m1 = Model.sarima(order=order, seasonal_order=seasonal_order)
m1.fit(y=y_train)
preds1 = m1.predict(h=H, x=None)

780 µs ± 27.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### StatsForecast

In [24]:
%%timeit
m2 = ARIMA(order=order, seasonal_order=seasonal_order[: -1], season_length=seasonal_order[-1])
preds2 = m2.forecast(y=y_train, h=H)["mean"]

3.92 s ± 12.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Statsmodels

In [25]:
%%timeit
m3 = (sm.tsa.statespace
      .SARIMAX(endog=y_train, order=order, seasonal_order=seasonal_order)
      .fit()
)
preds3 = m3.predict(start=y_train.size, end=y_train.size + H - 1)

16.9 s ± 1.56 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Plot

In [22]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df[DS], y=df[Y], mode='lines', name=Y))
test_dates = df[DS].values[-H:]
fig.add_trace(go.Scatter(x=test_dates, y=preds1, mode='lines', name="MyArima"))
fig.add_trace(go.Scatter(x=test_dates, y=preds2, mode='lines', name="StatsForecast"))
fig.add_trace(go.Scatter(x=test_dates, y=preds3, mode='lines', name="Statsmodels"))
fig.update_layout(showlegend=True)
fig.show()